<a href="https://colab.research.google.com/github/ZohebAbai/Tiny-ImageNet-Challenge/blob/master/Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing libraries and Downloading Dataset

In [0]:
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
## Download the dataset and unzip it 
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip

--2019-04-14 22:31:46--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  66.7MB/s    in 4.8s    

2019-04-14 22:31:51 (49.4 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [0]:
!unzip -qq 'tiny-imagenet-200.zip'
!ls

sample_data  tiny-imagenet-200	tiny-imagenet-200.zip


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install git+https://github.com/keras-team/keras-contrib.git

  Cloning https://github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-p9bsdlvh
  Stored in directory: /tmp/pip-ephem-wheel-cache-kyhxk74g/wheels/eb/42/ea/ef324c6958836b1a3d4c0659214b0bae1c0a9bf151254dc93f
Successfully built keras-contrib


In [0]:
# Importing important libraries
import numpy as np
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, SeparableConv2D
from keras.optimizers import Adam, RMSprop, SGD
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras_contrib.callbacks import CyclicLR
import imgaug as ia
from imgaug import augmenters as iaa

# random seed
np.random.seed(seed=101)
ia.seed(101)

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# input image dimensions
img_height = 64
img_width = 64
channels = 3

# Hyperparameters
batch_size = 128
num_classes = 200
epochs = 24
num_train = 100000
num_validation = 10000

In [0]:
val_data = pd.read_csv('./tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, 
                       names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)
val_data.head(3)

,File,Class
0,val_0.JPEG,n03444034
1,val_1.JPEG,n04067472
2,val_2.JPEG,n04070727


# Image Augmentation

In [0]:
# Defining Customized Imagedatagenerator using imgaug library
def CustomImageDataGen(input_img):
  # Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
  # e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second
  # image.
  sometimes = lambda aug: iaa.Sometimes(0.5, aug)
  
  seq = iaa.Sequential([
      iaa.Fliplr(0.5), # horizontal flips
      iaa.Flipud(0.2), # vertical flips
      
      # Small gaussian blur with random sigma between 0 and 0.5.
      # But we only blur about 50% of all images.
      sometimes(iaa.GaussianBlur(sigma=(0, 2.0))),
      
      # crop images by -10% to 20% of their height/width
      sometimes(iaa.CropAndPad(
          percent=(-0.1, 0.2),
          pad_mode=ia.ALL,
          pad_cval=(0, 255)
        )),
      
      # Apply affine transformations to some of the images
      # - scale to 80-120% of image height/width (each axis independently)
      # - translate by -20 to +20 relative to height/width (per axis)
      # - rotate by -45 to +45 degrees
      # - shear by -16 to +16 degrees
      # - order: use nearest neighbour or bilinear interpolation (fast)
      # - mode: use any available mode to fill newly created pixels
      #         see API or scikit-image for which modes are available
      # - cval: if the mode is constant, then use a random brightness
      #         for the newly created pixels (e.g. sometimes black,
      #         sometimes white)
      sometimes(iaa.Affine(
          scale={"x": (0.8, 1.5), "y": (0.8, 1.5)},
          translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
          rotate=(-45, 45),
          shear=(-16, 16),
          order=[0, 1],
          cval=(0, 255),
          mode=ia.ALL
      )),
      
      #drop 2-5% percent of the original size, leading to large dropped
      # rectangles.
      sometimes(iaa.CoarseDropout(
                        (0.03, 0.15), size_percent=(0.02, 0.05),
                        per_channel=0.2
                    )),
                
      # Make some images brighter and some darker.
      # In 20% of all cases, we sample the multiplier once per channel,
      # which can end up changing the color of the images.
      sometimes(iaa.Multiply((0.8, 1.2), per_channel=0.2)),
      
      #Improve or worsen the contrast of images.
      #Comment it out after third model run (extreme saturation)
      sometimes(iaa.ContrastNormalization((0.75, 1.5), per_channel=0.5)), 
     ],
     # do all of the above augmentations in random order
     random_order = True) # apply augmenters in random order
  
  output_img = seq.augment_image(input_img)
  return output_img

train_datagen = ImageDataGenerator(rescale=1/255., preprocessing_function = CustomImageDataGen)
valid_datagen = ImageDataGenerator(rescale=1/255.)

In [0]:
train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', 
                                                    target_size=(img_width, img_height), 
                                                    batch_size=batch_size, 
                                                    class_mode='categorical', 
                                                    shuffle=True, seed=101)

Found 100000 images belonging to 200 classes.


In [0]:
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', 
                                                         x_col='File', y_col='Class', 
                                                         target_size=(img_width, img_height),
                                                         class_mode='categorical', 
                                                         batch_size=batch_size, 
                                                         shuffle=False, seed=101)

Found 10000 images belonging to 200 classes.


# Model Building and Compilation

### Below is the custom Resnet Model, where architecture is inspired from Wide-Resnet and Resnet-18 keras models. As required for the project followings have not been used :
1. 1x1 for an increasing number of channels
2. dropout
3. fully connected layers
4. test dataset for training
5. pre-trained model/weights
6. someone else's code 

### Note : For running the new model after 8 hours, don't run beyond this, as the saved model contains model architecture, weights and optimizer.

In [0]:
# Model building
input = Input(shape=(img_height, img_width, channels))

# Block 1
layer0 = Conv2D(32, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(input)
layer0 = BatchNormalization()(layer0)
layer0 = Activation('relu')(layer0)

skip_connection_1 = layer0

# Block 2

layer1 = Conv2D(128, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer0)
layer1 = BatchNormalization()(layer1)
layer1 = Activation('relu')(layer1)

layer2 = Conv2D(128, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer1)
layer2 = BatchNormalization()(layer2)
layer2 = Activation('relu')(layer2)

layer3 = Conv2D(128, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer2)
layer3 = BatchNormalization()(layer3)
layer3 = Activation('relu')(layer3)

layer4 = Conv2D(128, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer3)
layer4 = BatchNormalization()(layer4)
layer4 = Activation('relu')(layer4)

layer5 = concatenate([skip_connection_1, layer4])
layer5 = BatchNormalization()(layer5)
layer5 = Activation('relu')(layer5)
layer5 = MaxPooling2D(pool_size=(2, 2))(layer5)

skip_connection_2 = layer5

# Block 3

layer6 = Conv2D(256, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer5)
layer6 = BatchNormalization()(layer6)
layer6 = Activation('relu')(layer6)

layer7 = Conv2D(256, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer6)
layer7 = BatchNormalization()(layer7)
layer7 = Activation('relu')(layer7)

layer8 = Conv2D(256, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer7)
layer8 = BatchNormalization()(layer8)
layer8 = Activation('relu')(layer8)

layer9 = Conv2D(256, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer8)
layer9 = BatchNormalization()(layer9)
layer9 = Activation('relu')(layer9)

layer10 = concatenate([skip_connection_2, layer9])
layer10 = BatchNormalization()(layer10)
layer10 = Activation('relu')(layer10)
layer10 = MaxPooling2D(pool_size=(2, 2))(layer10)

skip_connection_3 = layer10


# Block 4

layer11 = Conv2D(512, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer10)
layer11 = BatchNormalization()(layer11)
layer11 = Activation('relu')(layer11)

layer12 = Conv2D(512, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer11)
layer12 = BatchNormalization()(layer12)
layer12 = Activation('relu')(layer12)

layer13 = Conv2D(512, (3,3), padding='same',kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer12)
layer13 = BatchNormalization()(layer13)
layer13 = Activation('relu')(layer13)

layer14 = Conv2D(512, (3,3), padding='same',kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer13)
layer14 = BatchNormalization()(layer14)
layer14 = Activation('relu')(layer14)

layer15 = concatenate([skip_connection_3, layer14])
layer15 = BatchNormalization()(layer15)
layer15 = Activation('relu')(layer15)
layer15 = MaxPooling2D(pool_size=(2, 2))(layer15)


#Layer 16
layer16 = Conv2D(num_classes, (1,1), padding='same',kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer15)
layer16 = GlobalAveragePooling2D()(layer16)

#Output Layer
output = Activation('softmax')(layer16)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
# Model Summary
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 32)   896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 64, 64, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# Compile the Model
model.compile(loss='categorical_crossentropy',
              optimizer= RMSprop(lr= 0.0001, epsilon=1e-08),
              #optimizer= Adam(lr= 0.0001, epsilon=1e-08),
              #optimizer = SGD(momentum=0.9),
              metrics=['accuracy'])

In [0]:
# Callbacks
clr = CyclicLR(base_lr=0.0001, max_lr=0.0006, step_size=4686., mode='triangular2') #Cyclic learning rate
checkpointer = ModelCheckpoint(filepath="./drive/My Drive/Colab Notebooks/model1.hdf5", verbose=1, save_best_only=True, monitor="val_acc")

# Fit the Model
model.fit_generator(train_generator,
                    epochs=epochs,
                    steps_per_epoch= num_train // batch_size,
                    validation_steps= num_validation // batch_size,
                    validation_data=validation_generator,
                    verbose=1, callbacks=[clr, checkpointer]
                   )

Instructions for updating:
Use tf.cast instead.
Epoch 1/24
781/781 [==============================] - 1278s 2s/step - loss: 5.4892 - acc: 0.0531 - val_loss: 5.1116 - val_acc: 0.0896

Epoch 00001: val_acc improved from -inf to 0.08964, saving model to ./drive/My Drive/Colab Notebooks/model1.hdf5
Epoch 2/24
781/781 [==============================] - 1264s 2s/step - loss: 4.8541 - acc: 0.1075 - val_loss: 4.6112 - val_acc: 0.1277

Epoch 00002: val_acc improved from 0.08964 to 0.12774, saving model to ./drive/My Drive/Colab Notebooks/model1.hdf5
Epoch 3/24
781/781 [==============================] - 1261s 2s/step - loss: 4.5329 - acc: 0.1398 - val_loss: 4.2920 - val_acc: 0.1792

Epoch 00003: val_acc improved from 0.12774 to 0.17919, saving model to ./drive/My Drive/Colab Notebooks/model1.hdf5
Epoch 4/24
781/781 [==============================] - 1260s 2s/step - loss: 4.3436 - acc: 0.1672 - val_loss: 4.4600 - val_acc: 0.1747

Epoch 00004: val_acc did not improve from 0.17919
Epoch 5/24
781/78

### After 8 hours of running and saving the model, I shall run the new model from here after skipping the above part.

In [0]:
# load the model after 8 hours
from keras.models import load_model
new_model = load_model("./drive/My Drive/Colab Notebooks/model1.hdf5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
# callbacks
clr = CyclicLR(base_lr=0.00001, max_lr=0.00006, step_size=4686., mode='triangular2') #Cyclic learning rate
checkpointer_2 = ModelCheckpoint(filepath="./drive/My Drive/Colab Notebooks/model2.hdf5", verbose=1, save_best_only=True, monitor="val_acc")
# fit the model
new_model.fit_generator(train_generator,
                        epochs=epochs,
                        steps_per_epoch= num_train // batch_size,
                        validation_steps= num_validation // batch_size,
                        validation_data=validation_generator,
                        verbose=1, callbacks=[clr, checkpointer_2]
                       )

Epoch 1/24
781/781 [==============================] - 1192s 2s/step - loss: 4.1023 - acc: 0.5072 - val_loss: 3.7772 - val_acc: 0.5795

Epoch 00001: val_acc improved from -inf to 0.57953, saving model to ./drive/My Drive/Colab Notebooks/model2.hdf5
Epoch 2/24
781/781 [==============================] - 1180s 2s/step - loss: 4.0781 - acc: 0.5143 - val_loss: 3.7887 - val_acc: 0.5737

Epoch 00002: val_acc did not improve from 0.57953
Epoch 3/24
781/781 [==============================] - 1177s 2s/step - loss: 4.0725 - acc: 0.5165 - val_loss: 3.7946 - val_acc: 0.5775

Epoch 00003: val_acc did not improve from 0.57953
Epoch 4/24
781/781 [==============================] - 1177s 2s/step - loss: 4.0765 - acc: 0.5194 - val_loss: 3.8587 - val_acc: 0.5682

Epoch 00004: val_acc did not improve from 0.57953
Epoch 5/24
781/781 [==============================] - 1178s 2s/step - loss: 4.0900 - acc: 0.5178 - val_loss: 3.8729 - val_acc: 0.5688

Epoch 00005: val_acc did not improve from 0.57953
Epoch 6/24
7

### After 16 hours of running and saving the model, I shall run the new model from here after skipping the above parts.

### Before further running the model, we shall analyse the accuracy of individual classes

In [0]:
# load the model after 16 hours
from keras.models import load_model
extended_model = load_model("/content/drive/My Drive/Colab Notebooks/model2.hdf5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
#Prediction
pred=extended_model.predict_generator(validation_generator, steps= np.ceil(num_validation/batch_size), verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)

79/79 [==============================] - 39s 495ms/step


In [0]:
# Predicted class indices of 1st 10 val images
predicted_class_indices[:10]

array([107,  40,  89,  69, 132, 161,  99,  73, 145,  39])

In [0]:
# True class indices of 1st 10 val images
validation_generator.classes[:10]

[107, 139, 140, 69, 69, 161, 147, 73, 145, 39]

In [0]:
# Predicted classes from their indices
labels = (validation_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
predictions[:10]

['n03444034',
 'n02231487',
 'n03014705',
 'n02808440',
 'n03976657',
 'n04399382',
 'n03250847',
 'n02823428',
 'n04146614',
 'n02226429']

In [0]:
# Validation class names from words.txt
import os
class_to_name = dict()
file = open('tiny-imagenet-200/words.txt','r')
data= file.readlines()
for line in data:
  words = line.strip('\n').split('\t')
  class_to_name[words[0]] = words[1].split(',')[0]
file.close()

In [0]:
# Asserting Validation Class names from words.txt
validation_class_names={}
for _class in validation_generator.class_indices.keys():
  validation_class_names.update({_class : class_to_name[_class]})

In [0]:
# Classification Report of val classes
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(validation_generator.classes, predicted_class_indices,
                            #target_names=validation_generator.class_indices.keys(),
                            target_names=validation_class_names.values(),
                            digits=4))

                          precision    recall  f1-score   support

                goldfish     0.8431    0.8600    0.8515        50
European fire salamander     0.8571    0.8400    0.8485        50
                bullfrog     0.6111    0.6600    0.6346        50
             tailed frog     0.5532    0.5200    0.5361        50
      American alligator     0.6591    0.5800    0.6170        50
         boa constrictor     0.4510    0.4600    0.4554        50
               trilobite     0.6774    0.8400    0.7500        50
                scorpion     0.7500    0.5400    0.6279        50
             black widow     0.8039    0.8200    0.8119        50
               tarantula     0.6226    0.6600    0.6408        50
               centipede     0.7111    0.6400    0.6737        50
                   goose     0.7333    0.6600    0.6947        50
                   koala     0.7885    0.8200    0.8039        50
               jellyfish     0.7091    0.7800    0.7429        50
         

In [0]:
# Class Weight dictionary
"""class_weights = {0:1,1:1,2:1,3:1,4:1,5:1,6:1,7:1,8:1,9:1,10:1,
                 11:1,12:1,13:1,14:1,15:1,16:1,17:1,18:1,19:1,20:1,
                 21:1,22:1,23:1,24:1,25:1,26:1,27:1,28:1,29:1,30:1,
                 31:1,32:1,33:1,34:1,35:1,36:1,37:1,38:1,39:1,40:1,
                 41:1,42:1,43:1,44:1,45:1,46:1,47:1,48:1,49:1,50:1,
                 51:1,52:1,53:1,54:1,55:1,56:1,57:1,58:1,59:1,60:1,
                 61:1,62:1,63:1,64:1.2,65:1,66:1,67:1,68:1,69:1,70:1,
                 71:1,72:1,73:1,74:1,75:1,76:1,77:1,78:1,79:1,80:1.2,
                 81:1,82:1,83:1,84:1,85:1,86:1,87:1,88:1,89:1,90:1,
                 91:1,92:1,93:1,94:1,95:1,96:1,97:1,98:1,99:1,100:1,
                 101:1,102:1,103:1,104:1,105:1,106:1,107:1,108:1,109:1,110:1,
                 111:1,112:1,113:1,114:1,115:1,116:1,117:1,118:1,119:1,120:1,
                 121:1,122:1,123:1,124:1,125:1,126:1,127:1,128:1,129:1,130:1,
                 131:1.2,132:1,133:1,134:1,135:1,136:1,137:1,138:1,139:1,140:1,
                 141:1,142:1,143:1,144:1,145:1,146:1,147:1,148:1,149:1,150:1,
                 151:1,152:1,153:1,154:1,155:1,156:1,157:1,158:1,159:1,160:1,
                 161:1,162:1,163:1,164:1,165:1,166:1,167:1,168:1.2,169:1,170:1,
                 171:1,172:1.2,173:1,174:1,175:1.2,176:1,177:1,178:1,179:1,180:1,
                 181:1,182:1,183:1,184:1,185:1,186:1,187:1,188:1,189:1,190:1,
                 191:1,192:1,193:1,194:1,195:1,196:1,197:1,198:1,199:1,
                }"""

In [0]:
# Callbacks
clr = CyclicLR(base_lr=0.00001, max_lr=0.00006, step_size=3128., mode='triangular2') #Cyclic learning rate
checkpointer_3 = ModelCheckpoint(filepath="/content/drive/My Drive/Colab Notebooks/model3.hdf5", verbose=1, save_best_only=True, monitor="val_acc")
# fit the model
extended_model.fit_generator(train_generator,
                             epochs=epochs,
                             steps_per_epoch= np.ceil(num_train/batch_size),
                             validation_steps= np.ceil(num_validation/batch_size),
                             validation_data=validation_generator,
                             verbose=1, callbacks=[clr, checkpointer_3],
                             )

Epoch 1/24
782/782 [==============================] - 1233s 2s/step - loss: 4.0270 - acc: 0.5835 - val_loss: 3.9649 - val_acc: 0.6060

Epoch 00001: val_acc improved from -inf to 0.60600, saving model to /content/drive/My Drive/Colab Notebooks/model3.hdf5
Epoch 2/24
782/782 [==============================] - 1215s 2s/step - loss: 4.0542 - acc: 0.5800 - val_loss: 4.0233 - val_acc: 0.5931

Epoch 00002: val_acc did not improve from 0.60600
Epoch 3/24
782/782 [==============================] - 1215s 2s/step - loss: 4.0802 - acc: 0.5772 - val_loss: 4.0899 - val_acc: 0.5814

Epoch 00003: val_acc did not improve from 0.60600
Epoch 4/24
782/782 [==============================] - 1213s 2s/step - loss: 4.1185 - acc: 0.5719 - val_loss: 4.1946 - val_acc: 0.5678

Epoch 00004: val_acc did not improve from 0.60600
Epoch 5/24
782/782 [==============================] - 1213s 2s/step - loss: 4.1382 - acc: 0.5731 - val_loss: 4.1346 - val_acc: 0.5799

Epoch 00005: val_acc did not improve from 0.60600
Epoch

## ### After 24 hours of running and saving the model, I shall run the new model from here after skipping the above parts.

In [0]:
# load the model after 24 hours
from keras.models import load_model
extended_model = load_model("/content/drive/My Drive/Colab Notebooks/model3.hdf5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
# Callbacks
clr = CyclicLR(base_lr=0.000001, max_lr=0.000006, step_size=1564., mode='triangular2') #Cyclic learning rate
checkpointer_4 = ModelCheckpoint(filepath="/content/drive/My Drive/Colab Notebooks/model4.hdf5", verbose=1, save_best_only=True, monitor="val_acc")
# fit the model
extended_model.fit_generator(train_generator,
                             epochs=12,
                             steps_per_epoch= np.ceil(num_train/batch_size),
                             validation_steps= np.ceil(num_validation/batch_size),
                             validation_data=validation_generator,
                             verbose=1, callbacks=[clr, checkpointer_4]
                             )

Epoch 1/12
782/782 [==============================] - 1300s 2s/step - loss: 4.0547 - acc: 0.6351 - val_loss: 4.1352 - val_acc: 0.6216

Epoch 00001: val_acc improved from -inf to 0.62160, saving model to /content/drive/My Drive/Colab Notebooks/model4.hdf5
Epoch 2/12
782/782 [==============================] - 1275s 2s/step - loss: 4.0245 - acc: 0.6460 - val_loss: 4.1414 - val_acc: 0.6205

Epoch 00002: val_acc did not improve from 0.62160
Epoch 3/12
782/782 [==============================] - 1275s 2s/step - loss: 3.9912 - acc: 0.6541 - val_loss: 4.1333 - val_acc: 0.6227

Epoch 00003: val_acc improved from 0.62160 to 0.62270, saving model to /content/drive/My Drive/Colab Notebooks/model4.hdf5
Epoch 4/12
782/782 [==============================] - 1275s 2s/step - loss: 3.9535 - acc: 0.6650 - val_loss: 4.1327 - val_acc: 0.6214

Epoch 00004: val_acc did not improve from 0.62270
Epoch 5/12
782/782 [==============================] - 1276s 2s/step - loss: 3.9429 - acc: 0.6691 - val_loss: 4.1337 -

### After 24 hours of running and saving the model, I shall run the new model from here after skipping the above parts.

In [0]:
# load the model after 28 hours
from keras.models import load_model
extended_model = load_model("/content/drive/My Drive/Colab  Notebooks/model4.hdf5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
# Callbacks
clr = CyclicLR(base_lr=0.00001, max_lr=0.00006, step_size=1564., mode='triangular2') #Cyclic learning rate
checkpointer_5 = ModelCheckpoint(filepath="/content/drive/My Drive/Colab Notebooks/model5.hdf5", verbose=1, save_best_only=True, monitor="val_acc")
# fit the model
extended_model.fit_generator(train_generator,
                             epochs=12,
                             steps_per_epoch= np.ceil(num_train/batch_size),
                             validation_steps= np.ceil(num_validation/batch_size),
                             validation_data=validation_generator,
                             verbose=1, callbacks=[clr, checkpointer_5]
                             )

Epoch 1/12
782/782 [==============================] - 1189s 2s/step - loss: 4.0826 - acc: 0.6316 - val_loss: 4.3063 - val_acc: 0.5952

Epoch 00001: val_acc improved from -inf to 0.59520, saving model to /content/drive/My Drive/Colab Notebooks/model5.hdf5
Epoch 2/12
782/782 [==============================] - 1170s 1s/step - loss: 4.1577 - acc: 0.6179 - val_loss: 4.4593 - val_acc: 0.5657

Epoch 00002: val_acc did not improve from 0.59520
Epoch 3/12
782/782 [==============================] - 1171s 1s/step - loss: 4.1736 - acc: 0.6200 - val_loss: 4.3616 - val_acc: 0.5924

Epoch 00003: val_acc did not improve from 0.59520
Epoch 4/12
782/782 [==============================] - 1171s 1s/step - loss: 4.0826 - acc: 0.6468 - val_loss: 4.2524 - val_acc: 0.6140

Epoch 00004: val_acc improved from 0.59520 to 0.61400, saving model to /content/drive/My Drive/Colab Notebooks/model5.hdf5
Epoch 5/12
782/782 [==============================] - 1171s 1s/step - loss: 4.0466 - acc: 0.6586 - val_loss: 4.2914 -

### After 32 hours of running and saving the model, I shall run the new model from here after skipping the above parts.

In [0]:
# load the model after 32 hours
from keras.models import load_model
extended_model = load_model("/content/drive/My Drive/Colab Notebooks/model5.hdf5")

In [0]:
# Callbacks
clr = CyclicLR(base_lr=0.0000001, max_lr=0.0000006, step_size=1564., mode='triangular2') #Cyclic learning rate
checkpointer_6 = ModelCheckpoint(filepath="/content/drive/My Drive/Colab Notebooks/model6.hdf5", verbose=1, save_best_only=True, monitor="val_acc")
# fit the model
extended_model.fit_generator(train_generator,
                             epochs=12,
                             steps_per_epoch= np.ceil(num_train/batch_size),
                             validation_steps= np.ceil(num_validation/batch_size),
                             validation_data=validation_generator,
                             verbose=1, callbacks=[clr, checkpointer_6]
                             )

Epoch 1/12
782/782 [==============================] - 1212s 2s/step - loss: 4.0801 - acc: 0.6662 - val_loss: 4.2796 - val_acc: 0.6258

Epoch 00001: val_acc improved from -inf to 0.62580, saving model to /content/drive/My Drive/Colab Notebooks/model6.hdf5
Epoch 2/12
782/782 [==============================] - 1203s 2s/step - loss: 4.0799 - acc: 0.6668 - val_loss: 4.2786 - val_acc: 0.6258

Epoch 00002: val_acc did not improve from 0.62580
Epoch 3/12
782/782 [==============================] - 1201s 2s/step - loss: 4.0770 - acc: 0.6670 - val_loss: 4.2744 - val_acc: 0.6266

Epoch 00003: val_acc improved from 0.62580 to 0.62660, saving model to /content/drive/My Drive/Colab Notebooks/model6.hdf5
Epoch 4/12
782/782 [==============================] - 1178s 2s/step - loss: 4.0737 - acc: 0.6657 - val_loss: 4.2751 - val_acc: 0.6261

Epoch 00004: val_acc did not improve from 0.62660
Epoch 5/12
782/782 [==============================] - 1169s 1s/step - loss: 4.0584 - acc: 0.6707 - val_loss: 4.2744 -

# Top Validation Accuracy : 62.73%